https://www.pythonfordatascience.org/factorial-anova-python/

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from itertools import permutations, product, combinations
from scipy.stats import pearsonr, spearmanr

from itertools import permutations

In [39]:
#https://www.scribbr.com/statistics/two-way-anova/
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multitest as multi

In [40]:
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning, HessianInversionWarning, ValueWarning
# ignore these warning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=HessianInversionWarning)
warnings.filterwarnings("ignore", category=ValueWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

## Settings

In [41]:
analysis = "Fastcore"
#analysis = "iMAT"

#analysis = "gimme"
#analysis = "init"
#analysis = "tinit"

In [42]:
analysis_type = "FVA"
#analysis_type = "pFBA"

### Sum of squares type

In [43]:
ss_type = 3

if ss_type == 3:
    formula = "activity ~ C(gender, Sum) + C(genotype, Sum) + C(diet, Sum) + C(gender, Sum)*C(genotype, Sum) + C(gender, Sum)*C(diet, Sum) + C(genotype, Sum)*C(diet, Sum) + C(gender, Sum)*C(genotype, Sum)*C(diet, Sum)"
    #formula = "activity ~ C(gender, Sum) + C(genotype, Sum) + C(diet, Sum)"
else:
    formula = "activity ~ C(gender) + C(genotype) + C(diet) + C(gender)*C(genotype) + C(gender)*C(diet) + C(genotype)*C(diet) + C(gender)*C(genotype)*C(diet)"

### Number of reactions to observe 
Only keep the first `n_reactions` reactions (unless set to zero)

In [44]:
n_reactions = 0 # all reactions will be included
#n_reactions = 10

## Read the data

In [45]:
df = pd.read_csv("data\\"+analysis_type+"_"+analysis+".csv", sep=";")

In [46]:
models = list(df.columns[1:])
#models = list(map(lambda x: x.split("_")[1].split(".")[0], models))

In [47]:
df.columns = [df.columns[0]] + models

#### Convert values to float and replace nans with zeros

In [48]:
df[models] = df[models].astype(float)
df = df.fillna(0)

### Filter the reactions

#### Remove the reactions that are always the same

In [49]:
#df[models].eq(df[models].iloc[:, 0], axis=0).all(axis=1)

In [50]:
df = df[~df[models].eq(df[models].iloc[:, 0], axis=0).all(axis=1)]

In [51]:
#df = df.loc[~(df[df.columns[1:]]==0).all(axis=1)]
#df = df.loc[~(df[df.columns[1:]]==1).all(axis=1)]

In [52]:
df.shape

(5789, 37)

#### If `n_reactions` is not zero only retain first `n_reactions`

In [53]:
if n_reactions:
    df = df.head(n_reactions)

# Groups

## Grouping by genotype

In [54]:
# WT
genotype0 = ["GSM1405493","GSM1405505","GSM1405517", 
              "GSM1405489","GSM1405501","GSM1405513",
              "GSM1405485","GSM1405497","GSM1405509",
              "GSM1405494","GSM1405506","GSM1405518",
              "GSM1405490","GSM1405502","GSM1405514",
              "GSM1405486","GSM1405498","GSM1405510"]
# KO
genotype1 = ["GSM1405495","GSM1405507","GSM1405519",
              "GSM1405491","GSM1405503","GSM1405515",
              "GSM1405487","GSM1405499","GSM1405511",
              "GSM1405496","GSM1405508","GSM1405520",
              "GSM1405492","GSM1405504","GSM1405516",
              "GSM1405488","GSM1405500","GSM1405512"]
genotype = (genotype0, genotype1)

## Grouping by diet

In [55]:
# LFnC
diet0 = ["GSM1405485","GSM1405497","GSM1405509","GSM1405487","GSM1405499","GSM1405511",
         "GSM1405486","GSM1405498","GSM1405510","GSM1405488","GSM1405500","GSM1405512"]

# HFnC
diet1 = ["GSM1405489","GSM1405501","GSM1405513","GSM1405491","GSM1405503","GSM1405515",
         "GSM1405490","GSM1405502","GSM1405514","GSM1405492","GSM1405504","GSM1405516"]

# HFC
diet2 = ["GSM1405493","GSM1405505","GSM1405517","GSM1405495","GSM1405507","GSM1405519",
         "GSM1405494","GSM1405506","GSM1405518","GSM1405496","GSM1405508","GSM1405520"]

diet = (diet0, diet1, diet2)

## Grouping by gender

In [56]:
# F
gender0 = ["GSM1405493","GSM1405505","GSM1405517",
           "GSM1405489","GSM1405501","GSM1405513",
           "GSM1405485","GSM1405497","GSM1405509",
           "GSM1405495","GSM1405507","GSM1405519",
           "GSM1405491","GSM1405503","GSM1405515",
           "GSM1405487","GSM1405499","GSM1405511"]

# M
gender1 = ["GSM1405494","GSM1405506","GSM1405518",
           "GSM1405490","GSM1405502","GSM1405514",
           "GSM1405486","GSM1405498","GSM1405510",
           "GSM1405496","GSM1405508","GSM1405520",
           "GSM1405492","GSM1405504","GSM1405516",
           "GSM1405488","GSM1405500","GSM1405512"]

gender = (gender0, gender1)

## Groups

In [57]:
groups = {"genotype": genotype, "diet": diet, "gender": gender}
labels = {"genotype": ("WT","KO"), "diet": ("LFnC", "HFnC", "HFC"), "gender": ("F","M")}

## Retain only observed models

In [58]:
observed = gender0 + gender1

In [59]:
df = df[[df.columns[0]] + observed]

In [60]:
df.shape

(5789, 37)

# Organize the data

In [61]:
df2 = pd.melt(df, id_vars=["rxns"])
df2.columns = ['rxn', 'model', 'activity']

# already did this
## convert activities to float
#df2['activity'] = df2['activity'].str.replace(",",".")
#df2['activity'] = df2['activity'].astype(float)

## replace nans with zero
#df2['activity'] = df2['activity'].fillna(0)

In [62]:
for i,factor in enumerate(gender):
    df2.loc[df2['model'].isin(factor), 'gender'] = "group " + str(i)
    
for i,factor in enumerate(genotype):
    df2.loc[df2['model'].isin(factor), 'genotype'] = "group " + str(i)

for i,factor in enumerate(diet):
    df2.loc[df2['model'].isin(factor), 'diet'] = "group " + str(i)

    

In [63]:
rxns = df2.rxn.unique()
len(rxns)

5789

## Basic test

In [64]:
#test run
data = df2[df2.rxn == rxns[0]]
model = ols(formula, data=data).fit()
aov_table = sm.stats.anova_lm(model, typ=ss_type)
Fs = aov_table['F'].values[1:-1]
Ps = aov_table['PR(>F)'].values[1:-1]

categories = list(aov_table['PR(>F)'].index[1:-1])
categories

['C(gender, Sum)',
 'C(genotype, Sum)',
 'C(diet, Sum)',
 'C(gender, Sum):C(genotype, Sum)',
 'C(gender, Sum):C(diet, Sum)',
 'C(genotype, Sum):C(diet, Sum)',
 'C(gender, Sum):C(genotype, Sum):C(diet, Sum)']

In [65]:
aov_table

,sum_sq,df,F,PR(>F)
Intercept,222993.828036,1.0,1156.000083,8.154141e-22
"C(gender, Sum)",771.604886,1.0,4.000000,5.693985e-02
"C(genotype, Sum)",771.604886,1.0,4.000000,5.693985e-02
"C(diet, Sum)",1543.209771,2.0,4.000000,3.167635e-02
"C(gender, Sum):C(genotype, Sum)",771.604886,1.0,4.000000,5.693985e-02
"C(gender, Sum):C(diet, Sum)",1543.209771,2.0,4.000000,3.167635e-02
"C(genotype, Sum):C(diet, Sum)",1543.209771,2.0,4.000000,3.167635e-02
"C(gender, Sum):C(genotype, Sum):C(diet, Sum)",1543.209771,2.0,4.000000,3.167635e-02
Residual,4629.629314,24.0,NaN,NaN


In [66]:
F_obt = np.zeros((len(rxns), len(Fs)))
P_obt = np.zeros((len(rxns), len(Ps)))

In [67]:
for i, rxn in enumerate(rxns):
    data = df2[df2.rxn == rxn]
    model = ols(formula, data=data).fit()
    aov_table = sm.stats.anova_lm(model, typ=ss_type)
    Fs = aov_table['F'].values[1:-1]
    Ps = aov_table['PR(>F)'].values[1:-1]    
    
    F_obt[i,:] = Fs
    P_obt[i,:] = Ps
    
    if i%100 == 0:
        print(str(i)+"/"+str(len(rxns)))

0/5789
100/5789
200/5789
300/5789
400/5789
500/5789
600/5789
700/5789
800/5789
900/5789
1000/5789
1100/5789
1200/5789
1300/5789
1400/5789
1500/5789
1600/5789
1700/5789
1800/5789
1900/5789
2000/5789
2100/5789
2200/5789
2300/5789
2400/5789
2500/5789
2600/5789
2700/5789
2800/5789
2900/5789
3000/5789
3100/5789
3200/5789
3300/5789
3400/5789
3500/5789
3600/5789
3700/5789
3800/5789
3900/5789
4000/5789
4100/5789
4200/5789
4300/5789
4400/5789
4500/5789
4600/5789
4700/5789
4800/5789
4900/5789
5000/5789
5100/5789
5200/5789
5300/5789
5400/5789
5500/5789
5600/5789
5700/5789


### Output of the basic test

In [68]:
df_res = pd.DataFrame(P_obt)
df_res.columns = categories
df_res['rxn'] = rxns
df_res = df_res[[df_res.columns[-1]]+list(df_res.columns[:-1])]

In [70]:
df_res

,rxn,"C(gender, Sum)","C(genotype, Sum)","C(diet, Sum)","C(gender, Sum):C(genotype, Sum)","C(gender, Sum):C(diet, Sum)","C(genotype, Sum):C(diet, Sum)","C(gender, Sum):C(genotype, Sum):C(diet, Sum)"
0,10FTHF7GLUtl,0.056940,0.056940,0.031676,0.056940,0.031676,0.031676,0.031676
1,10FTHF7GLUtm,0.056940,0.056940,0.031676,0.056940,0.031676,0.031676,0.031676
2,10FTHFtl,0.056940,0.056940,0.031676,0.056940,0.031676,0.031676,0.031676
3,11DOCRTSLte,0.003272,0.115523,0.522669,0.422247,0.522669,0.522669,0.522669
4,11DOCRTSLtm,0.011994,0.007556,0.568011,0.055579,0.970532,0.989821,0.488101
5,11DOCRTSTRNte,0.045822,0.045822,0.039274,0.045822,0.039274,0.039274,0.039274
6,11DOCRTSTRNtr,0.008109,0.569080,0.024733,0.569080,0.024733,0.719796,0.719796
7,12DHCHOLabc,0.056940,0.056940,0.031676,0.056940,0.031676,0.031676,0.031676
8,12DHCHOLt,0.056940,0.056940,0.031676,0.056940,0.031676,0.031676,0.031676
9,12HARACHDte,0.056940,0.056940,0.031676,0.056940,0.031676,0.031676,0.031676


### Basic ANOVA format the results

In [68]:
df_res.columns = list(map(lambda x: x.replace(", Sum","").replace("):C(",",").replace("C(","p("), df_res.columns))

### Basic ANOVA FDR

In [69]:
df_resq = df_res.copy()
for c in df_resq.columns[1:]:
    df_resq[c] = multi.multipletests(df_resq[c], method = 'fdr_bh')[1]
df_resq.columns = list(map(lambda x: x.replace("p(","q("), df_res.columns))

### Save the results

In [71]:
df_res.to_csv("results_ANOVA\\"+analysis_type+"_"+analysis+"_basic_p.csv", index=False)
df_resq.to_csv("results_ANOVA\\"+analysis_type+"_"+analysis+"_basic_q.csv", index=False)

# Randomization test

In [27]:
N = 10000
C_F = np.zeros((len(rxns), len(Fs)))
observed = np.array(observed)

In [28]:
df_test = df2.copy()

In [29]:
for n in range(N):
    # generate permutations
    samp = np.random.permutation(range(len(observed)))
    gender_test = np.array((samp[:len(gender0)], samp[len(gender0):]))
    gender_test = observed[gender_test]

    samp = np.random.permutation(range(len(observed)))
    genotype_test = np.array((samp[:len(genotype0)], samp[len(genotype0):]))
    genotype_test = observed[genotype_test]

    samp = np.random.permutation(range(len(observed)))
    diet_test = np.array((samp[:len(diet0)], samp[len(diet0):len(diet0)+len(diet1)], samp[len(diet0)+len(diet1):]))
    diet_test = observed[diet_test]

    # prepare the dataframe with permuted factors
    for i,factor in enumerate(gender_test):
        df_test.loc[df_test['model'].isin(factor), 'gender'] = i

    for i,factor in enumerate(genotype_test):
        df_test.loc[df_test['model'].isin(factor), 'genotype'] = i

    for i,factor in enumerate(diet_test):
        df_test.loc[df_test['model'].isin(factor), 'diet'] = i  
    
    F_test = np.zeros((len(rxns), len(Fs)))
    
    # go through all the reactions and assess their F values
    for i, rxn in enumerate(rxns):
        data = df_test[df_test.rxn == rxn]
        model = ols(formula, data=data).fit()
        aov_table = sm.stats.anova_lm(model, typ=ss_type)
        Fs = aov_table['F'].values[1:-1]
        

        F_test[i,:] = Fs

        if i%100 == 0:
            print(str(n)+"/"+str(N)+"; "+str(i)+"/"+str(len(rxns)))
    
    C_F[F_obt > F_test] += 1


0/1000; 0/10
1/1000; 0/10
2/1000; 0/10
3/1000; 0/10
4/1000; 0/10
5/1000; 0/10
6/1000; 0/10
7/1000; 0/10
8/1000; 0/10
9/1000; 0/10
10/1000; 0/10
11/1000; 0/10
12/1000; 0/10
13/1000; 0/10
14/1000; 0/10
15/1000; 0/10
16/1000; 0/10
17/1000; 0/10
18/1000; 0/10
19/1000; 0/10
20/1000; 0/10
21/1000; 0/10
22/1000; 0/10
23/1000; 0/10
24/1000; 0/10
25/1000; 0/10
26/1000; 0/10
27/1000; 0/10
28/1000; 0/10
29/1000; 0/10
30/1000; 0/10
31/1000; 0/10
32/1000; 0/10
33/1000; 0/10
34/1000; 0/10
35/1000; 0/10
36/1000; 0/10
37/1000; 0/10
38/1000; 0/10
39/1000; 0/10
40/1000; 0/10
41/1000; 0/10
42/1000; 0/10
43/1000; 0/10
44/1000; 0/10
45/1000; 0/10
46/1000; 0/10
47/1000; 0/10
48/1000; 0/10
49/1000; 0/10
50/1000; 0/10
51/1000; 0/10
52/1000; 0/10
53/1000; 0/10
54/1000; 0/10
55/1000; 0/10
56/1000; 0/10
57/1000; 0/10
58/1000; 0/10
59/1000; 0/10
60/1000; 0/10
61/1000; 0/10
62/1000; 0/10
63/1000; 0/10
64/1000; 0/10
65/1000; 0/10
66/1000; 0/10
67/1000; 0/10
68/1000; 0/10
69/1000; 0/10
70/1000; 0/10
71/1000; 0/10
72

554/1000; 0/10
555/1000; 0/10
556/1000; 0/10
557/1000; 0/10
558/1000; 0/10
559/1000; 0/10
560/1000; 0/10
561/1000; 0/10
562/1000; 0/10
563/1000; 0/10
564/1000; 0/10
565/1000; 0/10
566/1000; 0/10
567/1000; 0/10
568/1000; 0/10
569/1000; 0/10
570/1000; 0/10
571/1000; 0/10
572/1000; 0/10
573/1000; 0/10
574/1000; 0/10
575/1000; 0/10
576/1000; 0/10
577/1000; 0/10
578/1000; 0/10
579/1000; 0/10
580/1000; 0/10
581/1000; 0/10
582/1000; 0/10
583/1000; 0/10
584/1000; 0/10
585/1000; 0/10
586/1000; 0/10
587/1000; 0/10
588/1000; 0/10
589/1000; 0/10
590/1000; 0/10
591/1000; 0/10
592/1000; 0/10
593/1000; 0/10
594/1000; 0/10
595/1000; 0/10
596/1000; 0/10
597/1000; 0/10
598/1000; 0/10
599/1000; 0/10
600/1000; 0/10
601/1000; 0/10
602/1000; 0/10
603/1000; 0/10
604/1000; 0/10
605/1000; 0/10
606/1000; 0/10
607/1000; 0/10
608/1000; 0/10
609/1000; 0/10
610/1000; 0/10
611/1000; 0/10
612/1000; 0/10
613/1000; 0/10
614/1000; 0/10
615/1000; 0/10
616/1000; 0/10
617/1000; 0/10
618/1000; 0/10
619/1000; 0/10
620/1000; 

In [30]:
C_F

array([[750., 751., 750., 751., 750., 750., 751.],
       [992., 980., 515., 912.,   1.,   0., 489.],
       [990., 982., 530., 927.,   2.,   1., 505.],
       [765., 765., 758., 766., 759., 758., 758.],
       [980., 830., 766., 371., 964., 465., 767.],
       [  4.,   0., 142., 988., 350., 361., 144.],
       [955., 358., 531., 338., 935., 530., 529.],
       [945., 954., 927., 958., 937., 941., 939.],
       [818., 791., 438., 822., 416., 406., 418.],
       [720., 921., 303., 297., 293., 967., 565.]])

### Output of the randomization test

In [31]:
p_F = C_F / N
df_res2 = pd.DataFrame(p_F)
df_res2.columns = categories
df_res2['rxn'] = rxns
df_res2 = df_res2[[df_res2.columns[-1]]+list(df_res2.columns[:-1])]

### Randomization test format the results

In [34]:
df_res2.columns = list(map(lambda x: x.replace(", Sum","").replace("):C(",",").replace("C(","p("), df_res2.columns))
df_res2

,rxn,p(gender),p(genotype),p(diet),"p(gender,genotype)","p(gender,diet)","p(genotype,diet)","p(gender,genotype,diet)"
0,10FTHFtm,0.750,0.751,0.750,0.751,0.750,0.750,0.751
1,11DOCRTSLte,0.992,0.980,0.515,0.912,0.001,0.000,0.489
2,11DOCRTSLtm,0.990,0.982,0.530,0.927,0.002,0.001,0.505
3,17AHPRGNLONEte,0.765,0.765,0.758,0.766,0.759,0.758,0.758
4,17AHPRGSTRNte,0.980,0.830,0.766,0.371,0.964,0.465,0.767
5,2AMADPTm,0.004,0.000,0.142,0.988,0.350,0.361,0.144
6,2OXOADOXm,0.955,0.358,0.531,0.338,0.935,0.530,0.529
7,2OXOADPt,0.945,0.954,0.927,0.958,0.937,0.941,0.939
8,34HPLFM,0.818,0.791,0.438,0.822,0.416,0.406,0.418
9,35CGMPtn,0.720,0.921,0.303,0.297,0.293,0.967,0.565


### Randomization test FDR

In [35]:
df_res2q = df_res2.copy()
for c in df_res2q.columns[1:]:
    df_res2q[c] = multi.multipletests(df_res2q[c], method = 'fdr_bh')[1]
df_res2q.columns = list(map(lambda x: x.replace("p(","q("), df_res2.columns))
df_res2q

,rxn,q(gender),q(genotype),q(diet),"q(gender,genotype)","q(gender,diet)","q(genotype,diet)","q(gender,genotype,diet)"
0,10FTHFtm,0.992,0.982,0.851111,0.988,0.964,0.947500,0.852222
1,11DOCRTSLte,0.992,0.982,0.851111,0.988,0.010,0.000000,0.852222
2,11DOCRTSLtm,0.992,0.982,0.851111,0.988,0.010,0.005000,0.852222
3,17AHPRGNLONEte,0.992,0.982,0.851111,0.988,0.964,0.947500,0.852222
4,17AHPRGSTRNte,0.992,0.982,0.851111,0.988,0.964,0.883333,0.852222
5,2AMADPTm,0.040,0.000,0.851111,0.988,0.832,0.883333,0.852222
6,2OXOADOXm,0.992,0.982,0.851111,0.988,0.964,0.883333,0.852222
7,2OXOADPt,0.992,0.982,0.927000,0.988,0.964,0.967000,0.939000
8,34HPLFM,0.992,0.982,0.851111,0.988,0.832,0.883333,0.852222
9,35CGMPtn,0.992,0.982,0.851111,0.988,0.832,0.967000,0.852222


### Save the results

In [36]:
df_res2.to_csv("results_ANOVA\\"+analysis_type+"_"+analysis+"_randomization_p.csv", index=False)
df_res2q.to_csv("results_ANOVA\\"+analysis_type+"_"+analysis+"_randomization_q.csv", index=False)